In [ ]:
"""
残差网络 ResNet
稠密网络 DenseNet
"""

## 问题由来
理论上，添加新的隐藏层，会扩展解空间，因此会降低训练误差。然而在实践中，添加过多的层后训练误差往往不降反升。
即使利用批量归一化带来的数值稳定性使训练深层模型更加容易，该问题仍然存在。

针对这一问题，何恺明等人提出了残差网络（ResNet） [1]。它在2015年的ImageNet图像识别挑战赛夺魁，并深刻影响了后来的深度神经网络的设计。

## 改进
ResNet最主要的改进是 输入跳过这卷积运算后直接加在最后的ReLU激活函数前。可以参考doc里的图。
DenseNet模仿ResNet，最大的不同点在于，输入并不和模块的输出相加，而是和输出级联（concat）成一个更高维的向量。

In [ ]:
# ResNet示例代码
class Residual(tf.keras.Model):
    def __init__(self, num_channels, use_1x1conv=False, strides=1, **kwargs):
        super(Residual, self).__init__(**kwargs)
        self.conv1 = layers.Conv2D(num_channels,
                                   padding='same',
                                   kernel_size=3,
                                   strides=strides)
        self.conv2 = layers.Conv2D(num_channels, kernel_size=3,padding='same')
        if use_1x1conv:
            self.conv3 = layers.Conv2D(num_channels,
                                       kernel_size=1,
                                       strides=strides)
        else:
            self.conv3 = None
        self.bn1 = layers.BatchNormalization()
        self.bn2 = layers.BatchNormalization()

    def call(self, X):
        Y = activations.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return activations.relu(Y + X) # 输入X 和 各种操作（含conv, batch norm等）后的输出 相加，再激活